In [1]:
import collections
import gzip
import heapq
import string

# Búsquedes

En aquesta pràctica posarem en pràctica el vist referent a les búsquedes.

## Word Ladder

En el primer lloc implementarem un joc senzill anomenat Word Ladder, amb les següents regles:

Se't donen dues paraules amb el mateix nombre de lletres (utilitzarem paraules de tres i quatre lletres), per exemple, DOG i CAT. El teu objectiu és transformar la primera paraula en la segona substituint una lletra de cada vegada per qualsevol altra lletra, sempre que el resultat sigui una paraula anglesa correcta. Per exemple, podríem canviar DOG per FOG, però no per GOG (no és una paraula anglesa correcta) o per GOD (es van canviar dues lletres). Per tant, una manera d'anar de DOG a CAT podria ser:

**DOG => COG => COT => CAT**

 Hi ha moltes maneres de fer-ho, una altra seria:
 
**DOG => HOG => HOT => HAT => CAT**

El joc consisteix a trobar la seqüència més curta de canvis que transformen la primera paraula en la segona. En aquest exercici, utilitzarem un diccionari anglès per comprovar si una paraula és correcta o no. El diccionari el tenim en el fitxer words34.txt.gz, que conté totes les paraules angleses de tres i quatre lletres, una per línia.

El joc es pot generalitzar a paraules de qualsevol longitud, però en aquest exercici ens limitarem a paraules de tres i quatre lletres.

Definirem tres mesures cost per a l'algorisme de cerca:

* **Distància**: el nombre de canvis de lletres que s'han fet fins ara: 0 per a la paraula inicial, 1 per a la segona paraula, etc.
    * El cost de **DOG => COG => COT => CAT** és 3.
* **Scrabble**: Associem a cada lletra un valor enter, que és el seu valor en el joc Scrabble
    * El cost d'utilitzar A, E, I, O, U, L, N, S, T o R és 1
    * El cost de D o G és 2
    * El cost de B, C, M o P és 3
    * El cost de F, H, V, W o Y és 4
    * El cost de K és 5
    * El cost de J o X és 8
    * El cost de Q o Z és 10.    
    * El cost de **DOG => COG => COT => CAT** és 3 + 1 + 1 = 5.
* **Raresa**: El tercer cost està basat en com de comú és una paraula després de la primera. La idea es que passar a una paraula molt rara deuria ser més car que passar a una molt comuna. Proporcionarem una llista de les paraules en anglès més frequents i la seva **freqüència relativa.**
    * El cost de la seqüència **DOG => JOG => JOT => COT => CAT** es calcula com 1+R per a cada pas on R és la mesura de com de rara és la paraula introduïda pel pas. Mirant el words34.txt.gz amb la comanda Unix zmore trobem entrades incloent les següents:
        ```
        jog  10.052741
        jot  11.177437
        cot  10.071238
        cat   6.886906
        ```
        així que el cost és 4 més la suma dels punts, o 42.188322.

### Que hem de fer?

#### Funcions de suport

1. Implementar una funció **read_words(filename)** que llegeixi el fitxer de paraules i retorni un conjunt Python amb les paraules que hi ha al fitxer i la seva freqüència relativa.


    Crea un conjunt anomenat `words` amb les paraules de `words34.txt.gz`


In [2]:
def read_words(filename):
    """Return a set of all words in filename."""

    paraules = []
    with gzip.open(filename, 'rt') as f:
        for line in f:
            paraula, freq = line.split()
            paraules.append((paraula, float(freq)))

    return set(paraules)


In [3]:
words = read_words('words34.txt.gz')

2. Implementar una funció **word2key(word)** que retorni una cadena de caràcters que serveixi com a clau per a la paraula word. La clau ha de ser una cadena de caràcters que tingui el mateix nombre de caràcters que word, però amb els caràcters ordenats alfabèticament. Per exemple, la clau de "cat" és "act" i la clau de "dog" és "dgo". Aquesta funció ens permetrà agrupar les paraules en el diccionari segons la seva clau, i així només caldrà comprovar si una paraula és correcta o no amb les paraules que tenen la mateixa clau.

In [4]:
def word2key(word):
    """Return a key for word that can be used to group all anagrams of word."""
    return ''.join(sorted(word))


word2key('cat')

'act'

3. Implementar una funció **build_dict(words)** que construeixi un diccionari Python que contingui com a clau les claus de les paraules i com a valor un conjunt Python amb les paraules que tenen aquesta clau. Per exemple, si el diccionari té la clau "act", el seu valor serà el conjunt amb les paraules "act", "cat" i "tac" i les seves respectives freqüències.

    Crea un diccionari de paraules i freqüències anomenat `words_dict`

In [5]:
def build_dict(words):
    """Return a dictionary that groups words by their key."""
    words_dict = collections.defaultdict(list)

    for word, freq in words:
        words_dict[word2key(word)].append((word, freq))

    return words_dict


words_dict = build_dict(words)

#### Definició del problema

4. Utilitzem la classe `Problema` per implementar la classe de problema `WordLadderDistancia`, definint els métodes necessaris per a fer-ho funcionar.
    Crea una instància del problema anomenada `wordladder_distancia`

In [6]:
class Problema(object):
    """Aquesta és la classe abstracta per a un problema formal. Una nova àrea crea una subclasse d'aquesta, sobrescrivint `accions` i `accio`, i potser altres mètodes.
    L'heurística per defecte és 0 i el cost d'acció per defecte és 1 per a tots els estats.
    Quan crees una instància d'una subclasse, especifica els estats `inicial` i `final`
    (o proporciona un mètode `es_resultat`) i potser altres arguments de paraula clau per a la subclasse."""

    def __init__(self, inicial=None, final=None, **kwds):
        self.__dict__.update(inicial=inicial, final=final, **kwds)

    def accions(self, state):         raise NotImplementedError

    def accio(self, state, action):   raise NotImplementedError

    def es_resultat(self, state):     return state == self.final

    def cost_accio(self, s, a, s1):   return 1

    def h(self, estat):               return 0

    def __str__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.inicial, self.final)

In [7]:
class WordLadderDistancia(Problema):
    def accions(self, state):
        for pos, orig_letter in enumerate(state):
            for letter in string.ascii_lowercase:
                opc = state[:pos] + letter + state[pos + 1:]
                if word2key(opc) in words_dict:
                    yield pos, letter, opc

    def cost_accio(self, s, a, s1):
        return 1

#### Algorismes de cerca no informada

6. Tenint en compte que el cost de cada acció serà **1** dedueix quin algorime hauries d'implementar per solucionar `wordladder_distancia`. Implementa'l i comprova que troba la solució correcta.

In [8]:
def breadth_first_search(problema):
    frontera = [(problema.inicial, 0, [])]
    visitats = set()

    i=0
    while len(frontera) > 0:
        i+=1
        estat, cost, ruta = frontera.pop(0)
        visitats.add(estat)

        if problema.es_resultat(estat):
            print(i)
            return estat, cost, ruta + [estat]
        for pos, letter, opcio in problema.accions(estat):
            if opcio not in visitats:
                frontera.append((opcio, cost + 1, ruta + [estat]))


In [9]:
wordladder_distancia = WordLadderDistancia(inicial="dog", final="cat")
breadth_first_search(wordladder_distancia)

3257
7495


('zap', 3, ['kit', 'zit', 'zip', 'zap'])

6. P: Quin algorisme has implementat? Per què?
    R: 


### Funcions de cost i heurístiques

7. Defineix una classe `WordLadderScrabble` que hereti de `WordLadderDistancia` i que implementi la funció de cost Scrabble. Crea una instància anomenada `wordladder_scrabble`.

In [10]:
class WordLadderScrabble(WordLadderDistancia):
    def cost_accio(self, s, a, s1):
        costos = {
            'a': 1, 'e': 1, 'i': 1, 'o': 1, 'u': 1,
            'l': 1, 'n': 1, 's': 1, 't': 1, 'r': 1,
            'd': 2, 'g': 2,
            'b': 3, 'c': 3, 'm': 3, 'p': 3,
            'f': 4, 'h': 4, 'v': 4, 'w': 4, 'y': 4,
            'k': 5,
            'j': 8, 'x': 8,
            'q': 10, 'z': 10,
        }
        return costos[a]

wordladder_scrabble = WordLadderScrabble(inicial="dog", final="cat")
breadth_first_search(wordladder_scrabble)

3257


('cat', 3, ['dog', 'cog', 'cot', 'cat'])

8. Implementa un algorisme que puga solucionar el problema `wordladder_scrabble` utilitzant la funció de cost Scrabble. Intenta buscar la solució per passar de "dog" a "cat".

In [11]:
def ucs(problema):
    frontera = []
    heapq.heappush(frontera, (0, problema.inicial, []))
    visitats = set()
    
    i=0
    while len(frontera) > 0:
        i+=1
        cost, estat, ruta = heapq.heappop(frontera)
        
        if i%1000 == 0:
            print(i, cost, estat, ruta)
            
        visitats.add(estat)
    
        if problema.es_resultat(estat):
            print(i)
            return estat, cost, ruta + [estat]
        
        for pos, letter, opcio in problema.accions(estat):
            if opcio not in visitats and len(ruta) <= len(problema.final):
                heapq.heappush(frontera, (cost + problema.cost_accio(estat, letter, opcio), opcio, ruta + [estat]))

wordladder_scrabble = WordLadderScrabble(inicial="dog", final="cat")
ucs(wordladder_scrabble)
                

1000 3 eot ['dog', 'dos', 'dot']
2000 3 sae ['dog', 'dag', 'sag']
3000 4 alr ['dog', 'aog', 'alg', 'ale']
4000 4 dro ['dog', 'dag', 'dal', 'dao']
5000 4 elu ['dog', 'eog', 'eag', 'eau']
6000 4 etl ['dog', 'dol', 'del', 'eel']
7000 4 iet ['dog', 'deg', 'ieg', 'ies']
8000 4 leu ['dog', 'deg', 'deo', 'deu']
9000 4 net ['dog', 'dos', 'des', 'det']
10000 4 oat ['dog', 'dag', 'aag', 'oag']
11000 4 ote ['dog', 'doa', 'dta', 'dte']
12000 4 ret ['dog', 'log', 'lot', 'let']
13000 4 sat ['dog', 'dag', 'oag', 'oat']
14000 4 sit ['dog', 'dig', 'dir', 'sir']
15000 4 tas ['dog', 'tog', 'ton', 'tan']
16000 4 tin ['dog', 'dig', 'tig', 'tie']
17000 4 tto ['dog', 'tog', 'too', 'tao']
18000 4 uit ['dog', 'eog', 'eot', 'eit']
19000 5 agt ['dog', 'dor', 'aor', 'agr']
19785


('cat', 5, ['dog', 'cog', 'cot', 'cat'])

9. Intenta buscar la solució per passar de "cold" a "warm". Pots trobar una solució? Per què?

In [12]:
#coldwarm_scrabble = DogCatScrabble(inicial="cold", final="warm")
#ucs(coldwarm_scrabble)

10. Implementa una classe `WordLadderScrabbleHeuristic` que hereti de `WordLadderScrabble` i que implementi la funció heurística basada en la suma dels costos de les lletres de la paraula actual fins a la paraula final. Crea una instància anomenada `wordladder_scrabble_heuristic`. Implementa un algorisme que puga solucionar el problema `wordladder_scrabble_heuristic`.
    Intenta buscar la solució per passar de "cold" a "warm".

In [13]:
class WordLadderScrabbleHeuristic(WordLadderScrabble):
    def h(self, estat):
        return sum(self.cost_accio(estat, letter, self.final[pos]) for pos, letter in enumerate(estat))

wordladder_scrabble_heuristic = WordLadderScrabbleHeuristic(inicial="cold", final="warm")

In [14]:
def a_star(problema):
    frontera = []
    heapq.heappush(frontera, (0, problema.inicial, []))
    visitats = set()
    
    i=0
    while len(frontera) > 0:
        i+=1
        cost, estat, ruta = heapq.heappop(frontera)
        
        if i%1000 == 0:
            print(i, cost, estat, ruta)
            
        visitats.add(estat)
    
        if problema.es_resultat(estat):
            print(i)
            return estat, cost, ruta + [estat]
        
        for pos, letter, opcio in problema.accions(estat):
            if opcio not in visitats:
                heapq.heappush(frontera, (cost + problema.cost_accio(estat, letter, opcio) + problema.h(opcio), opcio, ruta + [estat]))

In [15]:
#a_star(dogcat_scrabble_heuristic)

## Aplicació a altres problemes

Les técniques que hem vist per a resoldre el problema de Word Ladder es poden aplicar a altres problemes. En aquest exercici, les aplicarem al problema del puzzle de 8 peces.

### Puzzle de 8 peces

El puzzle de 8 peces és un joc que consisteix en un tauler de 3x3 en el que hi ha 8 peces i un espai buit. Com ja hem vist en les classes, podem representar el tauler com una llista de 9 enters, on el 0 representa l'espai buit. Per exemple, el tauler següent:

```
8 1 3
4 0 2
7 6 5
```

es pot representar com la llista `[8, 1, 3, 4, 0, 2, 7, 6, 5]`.

El joc consisteix en moure les peces horitzontalment o verticalment per a aconseguir un tauler amb les peces ordenades de menor a major, com el següent:

```
1 2 3
4 5 6
7 8 0
```

#### Que hem de fer?

1. Implementa la classe `Puzzle8Problema` que hereti de `Problema` i implementa els mètodes necessaris per a fer-ho funcionar. Crea una instància anomenada `puzzle8_distancia` amb el tauler inicial `[8, 1, 3, 4, 0, 2, 7, 6, 5]` i el tauler final `[1, 2, 3, 4, 5, 6, 7, 8, 0]`.

In [16]:
class Puzzle8Problema(Problema):
    def accions(self, state):
        for pos, orig_number in enumerate(state):
            if orig_number == 0:
                row, col = pos // 3, pos % 3
                if row > 0:
                    yield pos - 3, 'up', state[:pos - 3] + (0,) + state[pos - 2:pos] + (state[pos - 3],) + state[pos + 1:]
                if row < 2:
                    yield pos + 3, 'down', state[:pos] + (state[pos + 3],) + state[pos + 1:pos + 3] + (0,) + state[pos + 4:]
                if col > 0:
                    yield pos - 1, 'left', state[:pos - 1] + (0, state[pos - 1]) + state[pos + 1:]
                if col < 2:
                    yield pos + 1, 'right', state[:pos] + (state[pos + 1], 0) + state[pos + 2:]
                break

    def cost_accio(self, s, a, s1):
        return 1


2. Aplica l'algorisme de cerca en amplitud per a trobar una solució al problema `puzzle8_distancia`. Quina solució troba? Quin és el cost d'aquesta solució?

In [17]:
puzzle8_distancia = Puzzle8Problema(inicial=(8, 1, 3, 4, 0, 2, 7, 6, 5), final=(1, 2, 3, 4, 5, 6, 7, 8, 0))
breadth_first_search(puzzle8_distancia)

6923


((1, 2, 3, 4, 5, 6, 7, 8, 0),
 14,
 [(8, 1, 3, 4, 0, 2, 7, 6, 5),
  (8, 1, 3, 4, 2, 0, 7, 6, 5),
  (8, 1, 3, 4, 2, 5, 7, 6, 0),
  (8, 1, 3, 4, 2, 5, 7, 0, 6),
  (8, 1, 3, 4, 2, 5, 0, 7, 6),
  (8, 1, 3, 0, 2, 5, 4, 7, 6),
  (0, 1, 3, 8, 2, 5, 4, 7, 6),
  (1, 0, 3, 8, 2, 5, 4, 7, 6),
  (1, 2, 3, 8, 0, 5, 4, 7, 6),
  (1, 2, 3, 0, 8, 5, 4, 7, 6),
  (1, 2, 3, 4, 8, 5, 0, 7, 6),
  (1, 2, 3, 4, 8, 5, 7, 0, 6),
  (1, 2, 3, 4, 0, 5, 7, 8, 6),
  (1, 2, 3, 4, 5, 0, 7, 8, 6),
  (1, 2, 3, 4, 5, 6, 7, 8, 0)])

3.  Si implementem l'algorisme de cerca en cost uniforme, trobarà una solució millor? Perquè?

4. Implementa una classe `Puzzle8ProblemaHeuristic` que hereti de `Puzzle8Problema` i que implementi una funció heurística que estime el cost de moure les peces del tauler actual al tauler final. Crea una instància anomenada `puzzle8_distancia_heuristic` amb el tauler inicial `[8, 1, 3, 4, 0, 2, 7, 6, 5]` i el tauler final `[1, 2, 3, 4, 5, 6, 7, 8, 0]`.

In [18]:
class Puzzle8ProblemaHeuristic(Puzzle8Problema):
    def h(self, estat):
        return sum(abs(self.final.index(number) - estat.index(number)) for number in range(1, 9))

In [19]:
puzzle8_distancia_heuristic = Puzzle8ProblemaHeuristic(inicial=(8, 1, 3, 4, 0, 2, 7, 6, 5), final=(1, 2, 3, 4, 5, 6, 7, 8, 0))
a_star(puzzle8_distancia_heuristic)

428


((1, 2, 3, 4, 5, 6, 7, 8, 0),
 171,
 [(8, 1, 3, 4, 0, 2, 7, 6, 5),
  (8, 1, 3, 4, 2, 0, 7, 6, 5),
  (8, 1, 3, 4, 2, 5, 7, 6, 0),
  (8, 1, 3, 4, 2, 5, 7, 0, 6),
  (8, 1, 3, 4, 2, 5, 0, 7, 6),
  (8, 1, 3, 0, 2, 5, 4, 7, 6),
  (0, 1, 3, 8, 2, 5, 4, 7, 6),
  (1, 0, 3, 8, 2, 5, 4, 7, 6),
  (1, 2, 3, 8, 0, 5, 4, 7, 6),
  (1, 2, 3, 0, 8, 5, 4, 7, 6),
  (1, 2, 3, 4, 8, 5, 0, 7, 6),
  (1, 2, 3, 4, 8, 5, 7, 0, 6),
  (1, 2, 3, 4, 0, 5, 7, 8, 6),
  (1, 2, 3, 4, 5, 0, 7, 8, 6),
  (1, 2, 3, 4, 5, 6, 7, 8, 0)])